# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためにエージェントを実装することを目的としています。特に、エージェントアルファ（Agent Alpha）とエージェントベータ（Agent Beta）という2つのエージェントが設計されています。

### 取り組んでいる問題
エージェントは、20の質問ゲームの中で特定のキーワードを推測することに挑戦しています。最適な探索アルゴリズムとして、アルファベット順の比較と二分探索を用いて、最大20回のターン内に解決できるよう設計されています。エージェントは、他のチームメイトが正しく質問に答えることを前提としており、正規表現を用いたマッチングも採用されています。

### 使用している手法・ライブラリ
- **エージェントアルファ**: アルファベット順にキーワードを探索し、二分探索を通じて質問に対する回答を得ていきます。質問がハンドシェイクに基づく場合に特定の形で応答し、正規表現を使用して質問を解析します。
- **エージェントベータ**: プレースホルダーとしての役割を果たし、必要に応じて他の手法を応用することができます。
- **ライブラリ**: Pythonの標準ライブラリ（`random`, `re`）とKaggle Environmentsライブラリ（`kaggle_environments`）を使用して、ゲーム環境をシミュレーションしています。

### コードの構造
- **エージェント関数 (`agent_fn`)**: メインのプレイロジックを提供し、エージェントアルファが成功するまで試み、それが失敗した場合はエージェントベータを呼び出す設計になっています。
- **テスト用構成**: 環境の初期テストを行うための構成 (`debug_config`) が設けられています。

全体として、このノートブックは20の質問ゲームにおける効果的なエージェントの設計と実装に焦点を当てており、特定の探索手法を活用して成果を出すことを目指しています。

---


# 用語概説 
以下に、Jupyter Notebook内の専門用語の簡単な解説を列挙します。初心者向けではありますが、馴染みのない方に理解を助けるために、特に関連性の高い専門用語に焦点を当てています。

1. **エージェント (Agent)**:
   エージェントとは、特定のタスクを実行するために設計されたプログラムです。本コンペティションでは、質問者と回答者の役割を持つエージェントが対戦を行います。

2. **ハンドシェイクプロトコル (Handshake Protocol)**:
   通信において、接続を確立するための初期手続きのことを指します。このコンテキストでは、エージェントが正しく連携して動作するために、互いの状態を確認するためのプロトコルを紹介しています。

3. **正規表現 (Regular Expression)**:
   文字列のパターンを定義し、検索や置換などの操作を行うために使用される強力なツールです。本ノートブックでは、特定の質問パターンを認識するために利用されています。

4. **二分探索 (Binary Search)**:
   整列されたデータの中から特定の値を効率的に探すアルゴリズムで、データセットを半分に分けながら探索を進めます。このアルゴリズムにより、質問を通じて正解に迅速に近づくことができます。

5. **ターン (Turn)**:
   ゲームや対戦の一単位を指し、エージェントが質問、回答、推測を行う順番を示します。このコンペにおけるターン制のゲームプレイが重要な要素です。

6. **キーワード (Keywords)**:
   特定のコンテキストや質問に基づく重要な単語です。エージェントが推測するための候補となります。キーワードのリストを持つことで、エージェントはより戦略的に探索を行います。

7. **評価フェーズ (Evaluation Phase)**:
   各エージェントをテストし、その性能を評価するプロセスです。このフェーズでの成績が、エージェントの性能に関する指標に影響します。

8. **リーダーボード (Leaderboard)**:
   コンペティションの参加者やエージェントのスコアを表示し、現在の順位や成績を示すためのシステムです。

9. **試行錯誤 (Trial and Error)**:
   問題を解決するために、様々なアプローチを試しながら反復的に改善を目指す方法です。エージェントが最適な行動を見つけるためにこの方法を用いる場合があります。

10. **異常値 (Outlier)**:
    データセットの中で他のデータと著しく異なる値を指します。エージェントの推測や行動の中で、予期しない結果を引き起こす可能性があります。

これらの用語は、特に本ノートブックの文脈や機械学習・深層学習のアプローチを深く理解する上で重要な要素です。

---


In [ ]:
# 提出時に実行
# %mkdir /kaggle/working/submission

In [ ]:
# %%writefile submission/main.py


# エージェントアルファは、アルファベット順での比較と二分探索を使用。
# 最適な探索で、20回のターン以内に必ず解決策を見つける（log2(n_keywords)+1 の上限）
# 条件としては：
#   * 解決すべきキーワードが、以下のすべての単語のリストに含まれていること（2^19項目を超えない）
#   * 他のチームメイトもアルファを使っていること、すなわち正しく質問に答えることができること
#
# 解決すべきキーワードがリストにない場合、アルファ検索はキーワードを推測することはできませんが、他の技術と組み合わせることで結果が有用である場合があります。

# また、ハンドシェイクプロトコルを使用せずに回答者として受動的にプレイすることも可能です。しかし、質問者がハンドシェイクなしでアルファを使うことはあまり考えられません。とにかく、必要なのは、回答パイプラインの前に正規表現マッチャーがあることです。実装を参照してください。

# では、頑張ってください、loh-maa


import random
import re

VERSION = 9

# ゲームで期待されるすべてのキーワードのリスト。評価
# フェーズにエージェントを提出する前に更新する必要があります。
# 自分の判断を適用してください。リストが長いほどカバレッジが良くなりますが、収束が遅くなります。カバレッジは収束よりも重要であると言えるので、欠落している単語がない方が多くの余分なキーワードを持っている方がいいです。
allwords = ['xxx', 'remember', 'roll out your own!', 'xxx', 'this list is just for testing', 'advertisement',
            'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx', 'xxx',
            'agave', 'air compressor', 'air conditioner', 'air filter', 'air vent', 'alarm system',
            'analogy', 'anemone', 'anesthesia', 'apple pie', 'aprons', 'aquarium', 'atmosphere', 'auditorium',
            'backrest', 'bacteria', 'baguette', 'balcony', 'bank', 'barber chair', 'barcode', 'bat', 'bath mat',
            'battery bank', 'beanbag', 'bed frame', 'bike', 'bike path', 'binoculars', 'bird cage', 'bird seed',
            'bleachers', 'blinds', 'board games', 'bobby pins', 'bollards', 'bonding agent', 'bookcase', 'bookends',
            'bottled water', 'bread knife', 'bread pudding', 'break room', 'brewery merchandise', 'briefcase',
            'brochure', 'broken glass', 'brownies', 'bug spray', 'bulb', 'bumper sticker', 'bunsen burner', 'butterfly',
            'cabinet', 'calculator', 'cantaloupe', 'car seat', 'card', 'cardboard box', 'cash register', 'cat bed',
            'cat carrier', 'cauliflower', 'ceiling fan', 'cereal', 'latte', 'champagne flute', 'chandelier',
            'cheesecake', 'chessboard', 'chew toy', 'chocolate cake', 'cinnamon roll', 'rags', 'coat rack',
            'coffee beans', 'coffee grinder', 'coffee grounds', 'coffee makers', 'comic book', 'contact lenses',
            'conveyor belt', 'cooling tower', 'coral reefs', 'cream cheese', 'crochet hook', 'croissant', 'cup holder',
            'cupcake', 'curling iron', 'curtains', 'cutlery', 'cutting board', 'dandelion', 'deciduous tree',
            'dental chair', 'desk chairs', 'desk lamp', 'desktop computer', 'diaper', 'dijon mustard', 'dining table',
            'dish rack', 'dish soap', 'disinfectant', 'diving board', 'dog bed', 'dog crate', 'dog shampoo', 'donuts',
            'drain hose', 'drapes', 'duct tape', 'duffle bags', 'dumbbells', 'dump truck', 'duvet', 'dvd', 'dvd player',
            'dynamite', 'ear protection', 'earl grey tea', 'earplug', 'earth', 'edamame', 'edible flowers',
            'electric scooter', 'electric toothbrush', 'electrical outlet', 'electrical panel', 'electrical tape',
            'elevator', 'elliptical trainers', 'emergency exit sign', 'emergency lights', 'energy drink', 'engravings',
            'escalators', 'eucalyptus', 'excavator', 'exercise mat', 'exhaust fan', 'exit sign', 'extension cord',
            'eye mask', 'face mask', 'facial toner', 'fanny pack', 'fertilizer', 'filing cabinet', 'finger food',
            'fire alarm', 'fire escape ladder', 'fire extinguisher', 'fireplace', 'first aid kit', 'fish tank',
            'fishing pier', 'fitness tracker', 'flashlight', 'floor jacks', 'floor mats', 'foam roller', 'fog machine',
            'food bowl', 'food warmers', 'fortune cookie', 'frappuccino', 'free weights', 'french toast',
            'fridge magnet', 'fried rice', 'fungi', 'furniture', 'furniture polish', 'fuse', 'gadget', 'garage door',
            'garbage bag', 'garbage can', 'garbage disposal', 'garbage truck', 'gas mask', 'generator', 'glass table',
            'glove box', 'glove', 'golf cart', 'gps', 'grain bin', 'granola', 'grape vine', 'grapefruit',
            'graphic novel', 'graphing calculator', 'gravity', 'green beans', 'greeting card', 'guard tower',
            'guitar string', 'gym mats', 'habanero pepper', 'hair clip', 'hair dryer', 'hair gel', 'hairbrush',
            'hairspray', 'hamster wheel', 'hamstring curl machine', 'hand dryer', 'hand sanitizer', 'handbag',
            'handrail', 'hard hat', 'hash browns', 'hay bale', 'hazelnut', 'hdmi cable', 'headlamp', 'headphones',
            'hearing aid', 'heart rate monitors', 'heating element', 'hibiscus', 'high chair', 'highlighter',
            'hiking boots', 'holly', 'honeybee', 'hot dog', 'hot water bottle', 'ice cream maker', 'ice cube tray',
            'ice water', 'iced coffee', 'icicle', 'incense', 'incubator', 'index card', 'inhaler', 'ink cartridge',
            'insulation', 'router', 'interstate', 'iris', 'iron', 'ironing board', 'iron bar', 'irrigation system',
            'iv', 'jewelry box', 'journal', 'joystick', 'jumper cables', 'kale smoothie', 'karaoke machine',
            'key chain', 'khaki pants', 'kiosk', 'kitchen cabinet', 'garbage disposal', 'kitchen table',
            'komodo dragon', 'lab coat', 'lanyard', 'laptop', 'laser beam', 'lathe', 'laundry basket', 'lawnmower',
            'lego', 'library card', 'light switch', 'lightbulb', 'lint roller', 'lint trap', 'litter scooper',
            'lottery ticket', 'luggage', 'magazine rack', 'mailbox', 'marshmallow', 'masks', 'matches', 'measuring cup',
            'measuring tape', 'medical records', 'medication dispenser', 'medication', 'medicine cabinet', 'menorah',
            'metal detector', 'mouse', 'microphone stand', 'microscope', 'microwave', 'milling machine', 'mini fridge',
            'mixing bowl', 'mobile phone', 'modem', 'motorcycle helmet', 'movie poster', 'muffin tin', 'muffin',
            'muffler', 'mural', 'mushroom', 'nachos', 'nail clipper', 'nail polish', 'nail polish remover', 'name tag',
            'nap mat', 'necklace', 'nightstand', 'notebook', 'nozzle', 'oak table', 'ocarina', 'oil filter', 'ointment',
            'olive oil', 'onyx', 'orange peel', 'orange tree', 'orange zest', 'outdoor furniture', 'oven door',
            'oven rack', 'owl', 'oxygen tank', 'pacifier', 'packing peanuts', 'pad', 'pajamas', 'pallet jack',
            'pamphlet', 'pants', 'paper towels', 'paperweights', 'paperwork', 'parachute', 'parallel bars', 'pea soup',
            'pencil sharpener', 'perfume', 'personal protective equipment', 'pesticide', 'phone charger', 'photo album',
            'piano', 'pickaxe', 'piping tips', 'placemat', 'planter box', 'plaque', 'plastic gloves', 'plastic wrap',
            'plate', 'play mat', 'playing cards', 'pliers', 'polyester', 'pool cleaner', 'popcorn', 'popcorn machine',
            'portable speaker', 'pot holder', 'potato', 'power drill', 'power lines', 'power strip', 'press box',
            'pressure gauge', 'projector', 'protein bar', 'protein shakes', 'pry bar', 'public address system',
            'pumpkin pie', 'punching bag', 'push pin', 'puzzle', 'pyramid', 'quesadilla', 'rabbit', 'raccoon',
            'radio scanner', 'radish', 'railroad tracks', 'raindrops', 'rat', 'red velvet cake', 'red wine',
            'refrigerator', 'rehearsal room', 'reindeer', 'relish', 'reptile', 'resin', 'respirator', 'restaurants',
            'rice pudding', 'rivet', 'robot arm', 'rolling pin', 'roots', 'rowing machines', 'rug', 'rv',
            'safety harness', 'salad dressing', 'satellite', 'sauna', 'scones', 'scoreboard', 'scrap metal',
            'scratching posts', 'screwdriver', 'security systems', 'seed packet', 'server rack', 'sewing kit',
            'shin guards', 'shipping label', 'shoe rack', 'shower caddy', 'shower mat', 'sieve', 'silicone',
            'silicone mat', 'silver ore', 'sippy cup', 'sketchbook', 'sleeping bag', 'smartphone', 'smartwatch',
            'smoke detector', 'snacks', 'soap dish', 'soap dispenser', 'soap suds', 'soda', 'sofa', 'solar panel',
            'soldering iron', 'sound system', 'soybeans', 'space probe', 'spatula', 'spice rack', 'spider web',
            'sportswear', 'spotlight', 'sprinkler', 'sprinkler system', 'squirrel', 'stadium', 'stage lights',
            'stain remover', 'stained glass window', 'stair stepper', 'staircase', 'stationary bike', 'steam room',
            'sticker', 'sticky notes', 'storage bin', 'stove', 'street sign', 'sugar packet', 'suitcase', 'sunglasses',
            'sunscreen', 'swimsuit', 'swing set', 'tablet', 'tank', 'tape dispenser', 'taser', 'tea leaf', 'telescope',
            'television', 'tennis ball', 'tennis court', 'throw blanket', 'ticket machine', 'ticket stub', 'tire iron',
            'tissue box', 'toaster oven', 'tofu', 'toilet', 'toiletries bag', 'tomato', 'tongue depressor', 'tool belt',
            'tool box', 'toothbrush', 'toothbrush holder', 'toothpaste', 'touchscreen', 'tow truck', 'towel',
            'tracksuit', 'tractor shed', 'tractor', 'train brake', 'train door', 'trash bag', 'trash can', 'tricycle',
            'truck', 'tulip', 'turkey', 'turn signal', 'turnstiles', 'tweezers', 'udon noodles', 'ultraviolet lamp',
            'umbrella stand', 'underground river', 'unleavened bread', 'urinal', 'usb drives', 'utility box',
            'utility cart', 'vacuum cleaner', 'vanilla extract', 'vanity mirror', 'vans', 'vase', 'vegetable',
            'vegetation', 'veggie burger', 'velvet rope', 'vending machine', 'ventilation system', 'video camera',
            'violin bow', 'violin strings', 'virus', 'volcano', 'voucher', 'vr headset', 'walking stick', 'wall art',
            'wall decorations', 'wall street', 'wardrobe', 'washing machine', 'water', 'water bottle', 'water buffalo',
            'water fountain', 'water heater', 'water tank', 'watering can', 'webcam', 'wheelchair ramp', 'whistle',
            'willow tree', 'windbreak', 'wine aerator', 'wine decanter', 'wine opener', 'wine rack', 'wine tasting kit',
            'wire rack', 'wireless speaker', 'wrench', 'wrist weights', 'wristband', 'yoga block', 'yoga mat', 'yogurt',
            'zinc', 'afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua and barbuda', 'argentina',
            'armenia', 'australia', 'austria', 'azerbaijan', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium',
            'belize', 'benin', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'brazil', 'brunei',
            'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde',
            'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica',
            'croatia', 'cuba', 'cyprus', 'czech republic', 'democratic republic of the congo', 'denmark', 'djibouti',
            'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'england', 'equatorial guinea',
            'eritrea', 'estonia', 'ethiopia', 'federated states of micronesia', 'finland', 'france', 'gabon', 'gambia',
            'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 'guinea', 'guinea bissau', 'guyana',
            'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia', 'iran', 'iraq', 'ireland', 'israel',
            'italy', 'jamaica', 'japan', 'jordan', 'kazakhstan', 'kenya', 'kiribati', 'kosovo', 'kuwait', 'kyrgyzstan',
            'laos', 'latvia', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'lithuania', 'luxembourg',
            'madagascar', 'malawi', 'malaysia', 'maldives', 'mali', 'malta', 'marshall islands', 'mauritania',
            'mauritius', 'mexico', 'moldova', 'monaco', 'mongolia', 'montenegro', 'morocco', 'mozambique', 'myanmar',
            'namibia', 'nauru', 'nepal', 'netherlands', 'new zealand', 'nicaragua', 'niger', 'nigeria', 'north korea',
            'norway', 'oman', 'pakistan', 'palau', 'palestine', 'panama', 'papua new guinea', 'paraguay', 'peru',
            'philippines', 'poland', 'portugal', 'qatar', 'romania', 'russia', 'rwanda', 'saint kitts and nevis',
            'saint lucia', 'saint vincent and the grenadines', 'samoa', 'san marino', 'sao tome and principe',
            'saudi arabia', 'senegal', 'serbia', 'seychelles', 'sierra leone', 'singapore', 'slovakia', 'slovenia',
            'solomon islands', 'somalia', 'south africa', 'south korea', 'spain', 'sudan', 'suriname', 'swaziland',
            'sweden', 'switzerland', 'syria', 'taiwan', 'tajikistan', 'tanzania', 'thailand', 'togo', 'tonga',
            'trinidad and tobago', 'tunisia', 'turkey', 'turkmenistan', 'tuvalu', 'uganda', 'ukraine',
            'united arab emirates', 'united kingdom', 'united states of america', 'uruguay', 'uzbekistan', 'vanuatu',
            'venezuela', 'vietnam', 'yemen', 'zambia', 'zimbabwe', 'amsterdam netherlands', 'anaheim california',
            'austin texas', 'auckland new zealand', 'asheville north carolina', 'ashgabat turkmenistan']


def agent_beta(obs, cfg):
    """このエージェント関数はプレースホルダーです、自分自身のものを展開してください！LLMや好みの代替物を使用してください。"""

    if obs.turnType == 'ask':
        # LLMやその他をプレイ
        if len(obs.questions) % 2:
            response = "アルファは失敗しました、良いことにもう一つのエースがあります..."
        else:
            response = "私はあなたの個人的なAIアシスタントです、あなたの共同パイロットです！次の質問は何ですか？"

    elif obs.turnType == 'answer':
        response = 'はい'

    elif obs.turnType == 'guess':
        if len(obs.questions) % 2:
            # もしかしたら...愛らしいマーケティングで応答？
            responses = ['ノーアルファ、ノーファン', 'アルファをチェックした方がいいかも', 'アルファは人生だ',
                         'アルファベットの二分探索が助けてくれる', f'クリーミー＆ミルキー、アルファ {VERSION}',
                         'カグルを再び素晴らしく']

            response = random.choice(responses)

        else:
            # LLMやその他をプレイ
            response = "あなたの共同パイロットは近づいていると思います！**エッフェルタオル**"

    else:
        assert False

    return response


def answered_yes(obs, question):
    """質問が尋ねられ、答えが「はい」であったかどうかをチェックします。"""
    try:
        ix = obs.questions.index(question)
        return obs.answers[ix] == 'yes'
    except (ValueError, IndexError):
        return False


class AgentAlpha:

    # 互換性のために固定しておく
    HANDSHAKE = 'これはエージェントアルファですか？'

    # これが我々の検索空間で、変更されるため、連続テストの際には再初期化が必要です
    keywords = sorted(allwords)

    @staticmethod
    def parse_alpha_question(question):
        # アルファの質問に応答するための正規表現、以前のバージョンの4つのバリアントを含む：
        match = re.search(r'keyword.*(?:come before|precede) \"([^\"]+)\" .+ order\?$', question)
        if match:
            # アルファの質問にマッチ
            testword = match.group(1)
            return testword
        else:
            return None

    @staticmethod
    def play(obs, cfg):

        if obs.turnType == 'ask':

            if len(obs.questions) == 0:
                # これは最初の質問になる。
                return AgentAlpha.HANDSHAKE

            elif answered_yes(obs, AgentAlpha.HANDSHAKE) and AgentAlpha.keywords:
                # ただ1つの固定質問を使用します、もはや質問を回す意味はありません、
                # LLMは、いずれの質問にも信頼性がありません。
                testword = AgentAlpha.keywords[len(AgentAlpha.keywords) // 2]
                # 質問フォーマットを固定し、テストワードのみを置換します
                response = f'このキーワード（小文字）が、「{testword}」よりもアルファベット順に早いですか？'

            else:
                # キーワードが尽きました、解決策が取られていないか、回答者が間違った回答をしたため、
                # 他のアプローチを試せるようにNoneを返します
                response = None

        elif obs.turnType == 'answer':

            # ハンドシェイクの質問が見られた場合、はいと答えて、アルファの質問者が進行できるようにします。
            # そうでなければ、他の技術に頼ることになるかもしれませんが、アルファは最適なため、使うべきです。
            if AgentAlpha.HANDSHAKE == obs.questions[-1]:
                response = 'はい'

            else:
                testword = AgentAlpha.parse_alpha_question(obs.questions[-1])
                if testword is not None:
                    response = 'はい' if obs.keyword.lower() < testword else 'いいえ'
                else:
                    # アルファの質問ではないので、別のアプローチに進みます
                    response = None

        elif obs.turnType == 'guess':

            # 最後の質問がハンドシェイクであった場合、特別な推測をします
            if obs.questions[-1] == AgentAlpha.HANDSHAKE:
                # これはハンドシェイクラウンドです...
                if obs.answers[-1] == 'yes':
                    response = f"試してみましょう.. バージョン {VERSION}"
                else:
                    response = f'目'

            else:
                # 最後の質問と回答に基づいて空間を二分します
                testword = AgentAlpha.parse_alpha_question(obs.questions[-1])
                if testword:
                    if obs.answers[-1] == 'yes':
                        AgentAlpha.keywords = [k for k in AgentAlpha.keywords if k < testword]
                    else:
                        AgentAlpha.keywords = [k for k in AgentAlpha.keywords if not k < testword]

                if AgentAlpha.keywords:
                    # 前からポップしますが、どの側からポップしてもあまり重要ではありません
                    response = AgentAlpha.keywords.pop(0)

                else:
                    # キーワードが尽きた場合、他の手段を使います
                    response = None
        else:
            assert False, f'予期しないturnType: {obs.turnType}'

        return response


def agent_fn(obs, cfg):
    """メインフック、'agent_fn'という名前を保持し、提出の最後の関数にしてください。"""

    try:
        # エージェントアルファを最初に試みます、意味のあるプレイをするか
        # できない場合はNoneを返します
        response = AgentAlpha.play(obs, cfg)

        if response is None:
            # 他の方法を試すことができるように
            response = agent_beta(obs, cfg)

    except Exception:
        import traceback
        traceback.print_exc()
        response = 'いいえ'

    return response

In [ ]:
%%time

# テストのためのみ、提出時には無効にする
# （もし「提出ファイルが見つかりません」とエラーが出た場合）

import kaggle_environments


def agent_dummy(obs, cfg):
    if obs.turnType == "ask":
        response = "それはアヒルですか？"
    elif obs.turnType == "guess":
        response = "アヒル"
    elif obs.turnType == "answer":
        response = "いいえ"
    else:
        assert False
    return response

debug_config = {'episodeSteps': 61,  # 初期ステップとラウンドごとの3ステップ（質問/回答/推測）
                'actTimeout': 60,  # エージェントのラウンドごとの時間（秒）；デフォルトは60
                'runTimeout': 1200,  # エピソードの最大時間（秒）；デフォルトは1200
                'agentTimeout': 3600}  # 廃止されたフィールド；デフォルトは3600

env = kaggle_environments.make(environment="llm_20_questions", configuration=debug_config, debug=True)

# 単語が欠如していることをシミュレーション
AgentAlpha.keywords = sorted(random.sample(allwords, 500))

game_output = env.run(agents=[agent_dummy, agent_dummy, agent_fn, agent_fn])

env.render(mode="ipython", width=1080, height=700)
# out = env.render(mode="ansi")
# print(out)

In [ ]:
# 提出する際に有効にします.. pigzの使用は必須ですか？
# !apt install pigz pv
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

---

# コメント 

> ## OminousDude
> 
> あなたやあなたのコードに対して嫌悪感はありませんが、私はこのコードを使用することをお勧めしません。それはプライベートリーダーボードではほぼうまく機能しないからです。なぜならキーワードリスト、キーワードの種類、その他すべてが変わるからです。再度、嫌悪感や何もありません。良いコードとアイデアにアップボートしました！
> 
> 